In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('athlete_events.csv')
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [3]:
# filtering only values with 
df_med = df[~df['Medal'].isna()]
df_med

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
37,15,Arvo Ossian Aaltonen,M,30.0,NaN,NaN,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
38,15,Arvo Ossian Aaltonen,M,30.0,NaN,NaN,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
40,16,Juhamatti Tapio Aaltonen,M,28.0,184.0,85.0,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
41,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271078,135553,Galina Ivanovna Zybina (-Fyodorova),F,25.0,168.0,80.0,Soviet Union,URS,1956 Summer,1956,Summer,Melbourne,Athletics,Athletics Women's Shot Put,Silver
271080,135553,Galina Ivanovna Zybina (-Fyodorova),F,33.0,168.0,80.0,Soviet Union,URS,1964 Summer,1964,Summer,Tokyo,Athletics,Athletics Women's Shot Put,Bronze
271082,135554,Bogusaw Zych,M,28.0,182.0,82.0,Poland,POL,1980 Summer,1980,Summer,Moskva,Fencing,"Fencing Men's Foil, Team",Bronze
271102,135563,Olesya Nikolayevna Zykina,F,19.0,171.0,64.0,Russia,RUS,2000 Summer,2000,Summer,Sydney,Athletics,Athletics Women's 4 x 400 metres Relay,Bronze


In [6]:
import pandas as pd
import plotly.graph_objects as go

def plot_marimekko(dataframe, country):
    # Filter data for the specified country
    df_country = dataframe[dataframe['NOC'] == country]

    # Group by year and medal type and count the number of medals
    medal_counts = df_country.groupby(['Year', 'Medal']).size().reset_index(name='count')

    # Pivot the dataframe to have medal types as columns
    medal_pivot = medal_counts.pivot(index='Year', columns='Medal', values='count').fillna(0)

    # Calculate total medals per year and proportions for each medal type
    medal_pivot['total'] = medal_pivot.sum(axis=1)
    for medal in ['Gold', 'Silver', 'Bronze']:
        medal_pivot[f'{medal}_proportion'] = medal_pivot[medal] / medal_pivot['total']

    # Normalize widths based on the total medals to fit the graph size
    total_medals = medal_pivot['total'].sum()
    medal_pivot['width'] = medal_pivot['total'] / total_medals

    # Compute the x positions and cumulative proportions
    x_positions = [0]  # Starting position for the first bar
    for width in medal_pivot['width'][:-1]:
        x_positions.append(x_positions[-1] + width)
    medal_pivot['x'] = x_positions

    # Cumulative heights for stacked bar chart
    cumulative_gold = medal_pivot['Gold_proportion'].cumsum()
    cumulative_silver = (medal_pivot['Gold_proportion'] + medal_pivot['Silver_proportion']).cumsum()
    cumulative_bronze = (medal_pivot['Gold_proportion'] + medal_pivot['Silver_proportion'] + medal_pivot['Bronze_proportion']).cumsum()

    # Create the Marimekko chart
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=medal_pivot['x'],
        y=cumulative_bronze,
        width=medal_pivot['width'],
        marker=dict(color='brown'),
        name='Bronze',
        customdata=medal_pivot['total'],
        hovertemplate='Year: %{x}<br>Total Medals: %{customdata}<br>Proportion Bronze: %{y:.2f}<extra></extra>',
    ))

    fig.add_trace(go.Bar(
        x=medal_pivot['x'],
        y=cumulative_silver,
        width=medal_pivot['width'],
        marker=dict(color='silver'),
        name='Silver',
        customdata=medal_pivot['total'],
        hovertemplate='Year: %{x}<br>Total Medals: %{customdata}<br>Proportion Silver: %{y:.2f}<extra></extra>',
    ))

    fig.add_trace(go.Bar(
        x=medal_pivot['x'],
        y=cumulative_gold,
        width=medal_pivot['width'],
        marker=dict(color='gold'),
        name='Gold',
        customdata=medal_pivot['total'],
        hovertemplate='Year: %{x}<br>Total Medals: %{customdata}<br>Proportion Gold: %{y:.2f}<extra></extra>',
    ))

    # Update layout for the Marimekko chart
    fig.update_layout(
        title=f'Marimekko Chart of Medals for {country}',
        barmode='overlay',
        xaxis=dict(
            title='Year',
            tickmode='array',
            tickvals=medal_pivot['x'] + medal_pivot['width'] / 2,
            ticktext=medal_pivot.index,
        ),
        yaxis=dict(
            title='Proportion of Medals',
            tickformat='.0%',
        ),
    )

    fig.show()

# Example usage

df = pd.DataFrame(df)

plot_marimekko(df, 'USA')
